Here's what this notebook does, step-by-step:

1. Install the `montreal-force-aligner` (MFA) library using conda.
2. Download the Viet Bible Vox dataset from Huggingface.
3. Generate a normalized transcript file for each .mp3 file in the dataset.
4. Train an MFA model, then align speech and phonemes (creating a timestamp for each phoneme).
5. Generate a TensorFlow dataset (tfrecord files) that is ready for training TTS models.

In [1]:
# %pip install -Uqq tensorflow numpy torch tqdm librosa numba regex

In [2]:
# ./data is our working directory
# !mkdir -p data
%cd data

/home/billgates/lightspeed/data


In [3]:
#### INSTALL MFA  ####
# !wget https://repo.anaconda.com/miniconda/Miniconda3-py311_23.5.2-0-Linux-x86_64.sh -qO ./miniconda.sh
# # !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -qO $PWD/miniconda.sh # for Apple M1
# !bash ./miniconda.sh -b -p ./miniconda
# !source ./miniconda/bin/activate && conda create -n aligner -c conda-forge montreal-forced-aligner=2.2.15 -y --quiet
# if done install
# !source ./miniconda/bin/activate aligner

In [4]:
!pwd

/home/billgates/lightspeed/data


In [6]:
# !git lfs install # LFS is needed to download the data
# !git clone https://huggingface.co/datasets/ntt123/VietBibleVox
# !cd audio_1; unzip -qq audio_1.zip
!cd audio_1_experiment/audio_1_ver2; ls *.wav | wc -l
!cd audio_1_experiment/audio_1_ver2; ls *.txt | wc -l

2120
2120


In [17]:
import regex
import unicodedata
from pathlib import Path

In [18]:
vietnamese_characters = [
    'a', 'à', 'á', 'ả', 'ã', 'ạ',
    'ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ',
    'â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ',
    'e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ',
    'ê', 'ề', 'ế', 'ể', 'ễ', 'ệ',
    'i', 'ì', 'í', 'ỉ', 'ĩ', 'ị',
    'o', 'ò', 'ó', 'ỏ', 'õ', 'ọ',
    'ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ',
    'ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ',
    'u', 'ù', 'ú', 'ủ', 'ũ', 'ụ',
    'ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự',
    'y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ',
    'b', 'c', 'd', 'đ', 'g', 'h', 
    'k', 'l', 'm', 'n', 'p', 'q', 
    'r', 's', 't', 'v', 'x'
]
alphabet = "".join(vietnamese_characters)
space_re = regex.compile(r"\s+")
number_re = regex.compile("([0-9]+)")
digits = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
num_re = regex.compile(r"([0-9.,]*[0-9])")
keep_text_and_num_re = regex.compile(rf'[^\s{alphabet}.,0-9]')
keep_text_re = regex.compile(rf'[^\s{alphabet}]')

In [19]:
def read_number(num: str) -> str:
    if len(num) == 1:
        return digits[int(num)]
    elif len(num) == 2 and num.isdigit():
        n = int(num)
        end = digits[n % 10]
        if n == 10:
            return "mười"
        if n % 10 == 5:
            end = "lăm"
        if n % 10 == 0:
            return digits[n // 10] + " mươi"
        elif n < 20:
            return "mười " + end
        else:
            if n % 10 == 1:
                end = "mốt"
            return digits[n // 10] + " mươi " + end
    elif len(num) == 3 and num.isdigit():
        n = int(num)
        if n % 100 == 0:
            return digits[n // 100] + " trăm"
        elif num[1] == "0":
            return digits[n // 100] + " trăm lẻ " + digits[n % 100]
        else:
            return digits[n // 100] + " trăm " + read_number(num[1:])
    elif len(num) >= 4 and len(num) <= 6 and num.isdigit():
        n = int(num)
        n1 = n // 1000
        return read_number(str(n1)) + " ngàn " + read_number(num[-3:])
    elif "," in num:
        n1, n2 = num.split(",")
        return read_number(n1) + " phẩy " + read_number(n2)
    elif "." in num:
        parts = num.split(".")
        if len(parts) == 2:
            if parts[1] == "000":
                return read_number(parts[0]) + " ngàn"
            elif parts[1].startswith("00"):
                end = digits[int(parts[1][2:])]
                return read_number(parts[0]) + " ngàn lẻ " + end
            else:
                return read_number(parts[0]) + " ngàn " + read_number(parts[1])
        elif len(parts) == 3:
            return (
                read_number(parts[0])
                + " triệu "
                + read_number(parts[1])
                + " ngàn "
                + read_number(parts[2])
            )
    return num

In [20]:
def normalize_text(x):
    x = unicodedata.normalize('NFKC', x)
    x = x.lower()
    x = num_re.sub(r" \1 ", x)
    x = keep_text_and_num_re.sub(" ", x)
    words = x.split()
    words = [ read_number(w) if num_re.fullmatch(w) else w for w in words ]
    x = " ".join(words)
    x = keep_text_re.sub(" ", x)
    x = space_re.sub(" ", x)
    x = x.strip()
    return x

In [21]:
all_text = []
for fp in sorted(Path("audio_1_experiment/audio_1_ver2").glob("*.txt")):
    with open(fp, "r", encoding="utf-8") as f:
        text = f.read()
        # print(text)
        text = normalize_text(text)
        all_text.append(text)
    # override the text file
    with open(fp, "w", encoding="utf-8") as f:
        f.write(text)
all_words = sorted(set((" ".join(all_text)).split()))

In [22]:
with open("lexicon.txt", "w") as f:
    for w in all_words:
        w = w.strip()
        p = list(w)
        p = " ".join(p)
        f.write(f"{w}\t{p}\n")

In [23]:
%cd data

[Errno 2] No such file or directory: 'data'
/home/billgates/lightspeed/data


In [14]:
!source miniconda/bin/activate aligner; \
mfa train \
    --num_jobs `nproc` \
    --use_mp \
    --clean \
    --overwrite \
    --no_textgrid_cleanup \
    --single_speaker \
    --output_format json \
    --output_directory audio_1_experiment/audio_1_ver2 \
    audio_1_experiment/audio_1_ver2 ./lexicon.txt audio_1_experiment/audio_1_ver2_mfa

zsh:1: command not found: !mfa


Please be aware that you are running an alpha version of MFA. If you would like to install a more stable version, please visit https://montreal-forced-aligner.readthedocs.io/en/latest/installation.html#installing-older-versions-of-mfa
 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1,747/100  [ 0:00:00 < 0:00:00 , 9,147 it/s ]
 INFO     Found 1 speaker across 2120 files, average number of utterances per   
          speaker: 2120.0                                                       
 INFO     Initializing multiprocessing jobs...                                  
 INFO     Normalizing text...                                                   
  48% ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 1,027/2,120  [ 0:00:00 < -:--:-- , ? it/s ]^C
  48% ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 1,027/2,120  [ 0:00:00 < -:--:-- , ? it/s ]
 INFO     Done! Everything took 4.

In [ ]:
#### PREPARE TF DATASET ####
# Note: We are using a sampling rate of 16k,
# even though the original data has a sampling rate of 48k.

In [10]:
import json
from pathlib import Path
import numpy as np
import torch
import json
import librosa
import tensorflow as tf
from tqdm.auto import tqdm
import random

/home/billgates/miniconda3/envs/TTS_experiment_3/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-25 01:51:05.311984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-25 01:51:05.874372: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-25 01:51:05.874436: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library '

In [11]:
import json
from pathlib import Path
import numpy as np
import torch
import json
import librosa
import tensorflow as tf
from tqdm.auto import tqdm
import random

In [12]:
mel_basis = {}
hann_window = {}

def spectrogram_torch(y, n_fft, sampling_rate, hop_size, win_size, center=False):
    global hann_window
    if torch.min(y) < -1.0:
        print("min value is ", torch.min(y))
    if torch.max(y) > 1.0:
        print("max value is ", torch.max(y))

    dtype_device = str(y.dtype) + "_" + str(y.device)
    wnsize_dtype_device = str(win_size) + "_" + dtype_device
    if wnsize_dtype_device not in hann_window:
        hann_window[wnsize_dtype_device] = torch.hann_window(win_size).to(
            dtype=y.dtype, device=y.device
        )

    y = torch.nn.functional.pad(
        y.unsqueeze(1),
        (int((n_fft - hop_size) / 2), int((n_fft - hop_size) / 2)),
        mode="reflect",
    )
    y = y.squeeze(1)

    spec = torch.stft(
        y,
        n_fft,
        hop_length=hop_size,
        win_length=win_size,
        window=hann_window[wnsize_dtype_device],
        center=center,
        pad_mode="reflect",
        normalized=False,
        onesided=True,
        return_complex=True
    )
    spec = torch.view_as_real(spec)
    spec = torch.sqrt(spec.pow(2).sum(-1) + 1e-6)
    spec = spec.squeeze(0)
    return torch.swapaxes(spec, 0, 1)


def tensor_to_bytes(t):
    t = tf.constant(t)
    t = tf.io.serialize_tensor(t)
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[t.numpy()]))


def write_tfdata(data, out_file):
    with tf.io.TFRecordWriter(out_file) as file_writer:
        for wav_file, seq, _ in data:
            phone_seq = []
            for phone, duration in seq:
                phone_idx = phone_set.index(phone)
                phone_seq.append((phone_idx, duration))
            phone_seq = np.array(phone_seq, dtype=np.float32)

            # load wav
            wav, sr = librosa.load(wav_file, sr=config["data"]["sampling_rate"], dtype=np.float32)
            wav = torch.from_numpy(wav)
            # compute spec
            spec = spectrogram_torch(
                wav[None],
                n_fft=config["data"]["filter_length"],
                sampling_rate=config["data"]["sampling_rate"],
                hop_size=config["data"]["hop_length"],
                win_size=config["data"]["win_length"],
                center=False
            )
            features = {
                "phone_idx": tensor_to_bytes(phone_seq[:, 0].astype(np.int32)),
                "phone_duration": tensor_to_bytes(phone_seq[:, 1]),
                "wav": tensor_to_bytes(wav.half().numpy()),
                "spec": tensor_to_bytes(spec.half().numpy())
            }
            example = tf.train.Example(features=tf.train.Features(feature=features))
            file_writer.write(example.SerializeToString())

def write_split(split, data, num_chunks):
    data = np.array(data, dtype=object)
    chunks = list(np.array_split(data, num_chunks))
    for i, chunk in enumerate(tqdm(chunks)):
        write_tfdata(chunk, f"tfdata/{split}/part_{i:03d}.tfrecords")

In [13]:
!mkdir -p tfdata/{train,test}

In [5]:
%cd data

[Errno 2] No such file or directory: 'data'
/home/billgates/lightspeed/data


In [13]:
with open("../config.json", "rb") as f:
    config = json.load(f)
device = "cuda" if torch.cuda.is_available() else "cpu"
data_dir = Path("audio_1_experiment/audio_1/")
json_files = sorted(data_dir.glob("*.json"))
dataset = []
phone_set = []

for file_path in json_files:
    with open(file_path, "rb") as f:
        data = json.load(f)
    seq = []
    word_index = 0
    words =  data["tiers"]["words"]["entries"]
    for start, end, phone in data["tiers"]["phones"]["entries"]:
        if start > words[word_index][1] - 1e-5:
            seq.append( ("<SEP>", 0) )
            word_index += 1
        duration = end * 1000 - start * 1000 # ms
        phone_set.append(phone)
        seq.append( (phone, duration) )
    print("seq", seq)
    wav_file = file_path.with_suffix(".wav")
    dataset.append((wav_file, seq, data["end"]))

phone_set = ["<SEP>"] + sorted(set(phone_set))
assert len(phone_set) <= 256
with open("phone_set.json", "w", encoding="utf-8") as f:
    json.dump(phone_set, f)

assert phone_set.index("<SEP>") == 0

seq [('đ', 10.0), ('ọ', 10.0), ('c', 10.0), ('<SEP>', 0), ('t', 60.0), ('á', 100.0), ('c', 30.0), ('<SEP>', 0), ('p', 20.0), ('h', 50.0), ('ẩ', 150.0), ('m', 110.0), ('<SEP>', 0), ('sil', 150.0), ('<SEP>', 0), ('c', 40.0), ('h', 90.0), ('ê', 80.0), ('n', 80.0), ('h', 10.0), ('<SEP>', 0), ('sil', 110.0), ('<SEP>', 0), ('a', 100.0), ('n', 130.0), ('d', 20.0), ('<SEP>', 0), ('sil', 50.0), ('<SEP>', 0), ('s', 30.0), ('<SEP>', 0), ('sil', 390.0), ('<SEP>', 0), ('i', 179.99999999999977), ('t', 20.0), ('c', 10.000000000000227), ('h', 10.0), ('<SEP>', 0), ('sil', 100.0), ('<SEP>', 0), ('m', 90.0), ('ư', 40.0), ('ờ', 30.0), ('i', 30.0), ('<SEP>', 0), ('s', 130.0), ('á', 70.0), ('u', 130.0), ('<SEP>', 0), ('n', 30.0), ('g', 10.0), ('u', 30.0), ('y', 50.0), ('ê', 40.0), ('n', 70.0), ('<SEP>', 0), ('t', 60.0), ('ắ', 110.0), ('c', 90.0), ('<SEP>', 0), ('sil', 40.0), ('<SEP>', 0), ('n', 90.0), ('g', 10.0), ('h', 30.0), ('ĩ', 140.0), ('<SEP>', 0), ('sil', 50.0), ('<SEP>', 0), ('g', 10.0), ('i', 50.0)

In [12]:
random.Random(42).shuffle(dataset)
L = len(dataset) - 256
train_data = dataset[:L]
test_data = dataset[L:]
print("Train data size:", len(train_data))
print("Test data size:", len(test_data))

Train data size: 1858
Test data size: 256


In [8]:
%pwd

'/home/billgates/lightspeed/data'

In [10]:
train_data[0]

(PosixPath('audio_1_experiment/audio_1/audio_1-1600.wav'),
 [('n', 10.0),
  ('g', 10.0),
  ('h', 10.0),
  ('i', 10.0),
  ('ê', 10.0),
  ('m', 10.0),
  ('<SEP>', 0),
  ('c', 10.0),
  ('ủ', 10.0),
  ('a', 30.0),
  ('<SEP>', 0),
  ('m', 10.0),
  ('ộ', 10.0),
  ('t', 20.0),
  ('<SEP>', 0),
  ('n', 40.0),
  ('h', 50.0),
  ('à', 10.0),
  ('<SEP>', 0),
  ('sil', 170.0),
  ('<SEP>', 0),
  ('q', 10.0),
  ('u', 70.0),
  ('ý', 130.0),
  ('<SEP>', 0),
  ('t', 60.0),
  ('ộ', 160.0),
  ('c', 30.0),
  ('<SEP>', 0),
  ('sil', 80.0),
  ('<SEP>', 0),
  ('c', 40.0),
  ('ũ', 80.0),
  ('n', 70.0),
  ('g', 10.0),
  ('<SEP>', 0),
  ('c', 30.0),
  ('h', 40.0),
  ('ỉ', 260.0),
  ('<SEP>', 0),
  ('sil', 50.0),
  ('<SEP>', 0),
  ('d', 40.0),
  ('ự', 120.0),
  ('<SEP>', 0),
  ('đ', 60.0),
  ('ị', 90.0),
  ('n', 140.0),
  ('h', 10.0),
  ('<SEP>', 0),
  ('b', 50.0),
  ('a', 100.0),
  ('n', 120.0),
  ('<SEP>', 0),
  ('c', 30.0),
  ('h', 50.0),
  ('o', 160.0),
  ('<SEP>', 0),
  ('sil', 640.0),
  ('<SEP>', 0),
  ('â',

In [48]:
write_split("test", test_data, 1)

  0%|          | 0/1 [00:00<?, ?it/s]

min value is  tensor(-1.0057)
max value is  tensor(1.0066)
min value is  tensor(-1.0134)
max value is  tensor(1.0007)
min value is  tensor(-1.0017)
min value is  tensor(-1.0018)
max value is  tensor(1.0074)
min value is  tensor(-1.0090)
max value is  tensor(1.0072)
min value is  tensor(-1.0236)
max value is  tensor(1.0643)
max value is  tensor(1.0050)
min value is  tensor(-1.0191)
max value is  tensor(1.0407)
min value is  tensor(-1.1917)
max value is  tensor(1.2890)
min value is  tensor(-1.0175)
max value is  tensor(1.0168)
min value is  tensor(-1.0973)
max value is  tensor(1.1221)
max value is  tensor(1.0031)
max value is  tensor(1.1346)
min value is  tensor(-1.0115)
max value is  tensor(1.0135)
min value is  tensor(-1.0125)
min value is  tensor(-1.0056)
max value is  tensor(1.0150)
min value is  tensor(-1.0112)
max value is  tensor(1.0628)
min value is  tensor(-1.0069)
min value is  tensor(-1.0259)
max value is  tensor(1.0297)
min value is  tensor(-1.0111)
max value is  tensor(1.012

100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


In [49]:
write_split("train", train_data, 256)

  2%|▏         | 4/256 [00:00<00:06, 37.99it/s]

min value is  tensor(-1.0102)
max value is  tensor(1.0078)
min value is  tensor(-1.0127)
max value is  tensor(1.0073)
max value is  tensor(1.0001)
min value is  tensor(-1.1619)
max value is  tensor(1.0106)
min value is  tensor(-1.0390)
max value is  tensor(1.0229)
min value is  tensor(-1.0507)
max value is  tensor(1.0775)
min value is  tensor(-1.0139)
max value is  tensor(1.0135)
min value is  tensor(-1.0170)
max value is  tensor(1.0146)
min value is  tensor(-1.0963)
max value is  tensor(1.0154)
min value is  tensor(-1.0192)
max value is  tensor(1.0044)
min value is  tensor(-1.0127)
max value is  tensor(1.0032)
min value is  tensor(-1.0143)
min value is  tensor(-1.0103)
min value is  tensor(-1.0490)
max value is  tensor(1.0170)
min value is  tensor(-1.0140)
max value is  tensor(1.0102)
min value is  tensor(-1.0026)
max value is  tensor(1.1513)
min value is  tensor(-1.0233)
max value is  tensor(1.0061)
min value is  tensor(-1.0237)
min value is  tensor(-1.0131)


  5%|▌         | 14/256 [00:00<00:06, 39.20it/s]

min value is  tensor(-1.0352)
max value is  tensor(1.0223)
min value is  tensor(-1.0375)
max value is  tensor(1.1178)
min value is  tensor(-1.0082)
min value is  tensor(-1.1587)
max value is  tensor(1.0370)
min value is  tensor(-1.0617)
max value is  tensor(1.0422)
min value is  tensor(-1.0073)
min value is  tensor(-1.0024)
min value is  tensor(-1.0136)
max value is  tensor(1.0082)
min value is  tensor(-1.0104)
max value is  tensor(1.0021)
min value is  tensor(-1.0258)
max value is  tensor(1.0138)
min value is  tensor(-1.0404)
max value is  tensor(1.0162)
min value is  tensor(-1.0109)
max value is  tensor(1.0047)
min value is  tensor(-1.0156)
max value is  tensor(1.0046)
min value is  tensor(-1.0017)
min value is  tensor(-1.0146)
max value is  tensor(1.0060)
max value is  tensor(1.0022)
min value is  tensor(-1.0182)
max value is  tensor(1.0133)
min value is  tensor(-1.1985)
max value is  tensor(1.0343)
min value is  tensor(-1.1945)
max value is  tensor(1.0973)
min value is  tensor(-1.0

  9%|▊         | 22/256 [00:00<00:06, 38.93it/s]

...

min value is  tensor(-1.0377)
max value is  tensor(1.0390)
min value is  tensor(-1.0044)
max value is  tensor(1.0150)
min value is  tensor(-1.0063)
max value is  tensor(1.0010)
min value is  tensor(-1.0793)
max value is  tensor(1.0008)
min value is  tensor(-1.1070)
max value is  tensor(1.0821)
min value is  tensor(-1.0553)
max value is  tensor(1.1352)
min value is  tensor(-1.0062)
max value is  tensor(1.0045)


 98%|█████████▊| 251/256 [00:10<00:00, 24.45it/s]

min value is  tensor(-1.0246)
max value is  tensor(1.0062)
min value is  tensor(-1.0036)
min value is  tensor(-1.0607)
max value is  tensor(1.0063)
min value is  tensor(-1.0222)
max value is  tensor(1.0117)
min value is  tensor(-1.0112)
min value is  tensor(-1.1409)
max value is  tensor(1.2952)
min value is  tensor(-1.0472)
max value is  tensor(1.1664)
min value is  tensor(-1.0189)
max value is  tensor(1.0334)
min value is  tensor(-1.0089)
max value is  tensor(1.0153)


100%|██████████| 256/256 [00:10<00:00, 24.55it/s]

min value is  tensor(-1.0014)
min value is  tensor(-1.0156)
max value is  tensor(1.0056)
min value is  tensor(-1.0310)
max value is  tensor(1.0169)
min value is  tensor(-1.0112)
max value is  tensor(1.0022)
max value is  tensor(1.0048)
min value is  tensor(-1.0766)
max value is  tensor(1.0852)
min value is  tensor(-1.0184)
min value is  tensor(-1.0166)
max value is  tensor(1.0942)
min value is  tensor(-1.0152)
max value is  tensor(1.0478)
